In [1]:
import pandas as pd 


In [3]:
df = pd.read_csv(r"C:\Users\Administrator\Desktop\KPIM\Internship Folder\Dataset\DataCoSupplyChainDataset.csv", encoding='ISO-8859-1')

In [4]:
# Set start_date
category = category.withColumn("start_date", F.current_timestamp())




# Define a window specification
window_spec_key = Window.orderBy("category_id")  
# Add a new column 'product_key' with consecutive IDs starting from 1
category = category.withColumn("category_key", row_number().over(window_spec_key))



# Define window specification
windowSpec = Window.partitionBy("category_id").orderBy("start_date")

# Set end_date
category = category.withColumn("end_date", F.to_timestamp(F.lit("9999-12-31")))


category = category.withColumn("is_valid", lit(True))



,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,...,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Image,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,2/3/2018 22:56,Standard Class
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/18/2018 12:27,Standard Class
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/17/2018 12:06,Standard Class
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/16/2018 11:45,Standard Class
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/15/2018 11:24,Standard Class


In [5]:
df.dtypes

Type                              object
Days for shipping (real)           int64
Days for shipment (scheduled)      int64
Benefit per order                float64
Sales per customer               float64
Delivery Status                   object
Late_delivery_risk                 int64
Category Id                        int64
Category Name                     object
Customer City                     object
Customer Country                  object
Customer Email                    object
Customer Fname                    object
Customer Id                        int64
Customer Lname                    object
Customer Password                 object
Customer Segment                  object
Customer State                    object
Customer Street                   object
Customer Zipcode                 float64
Department Id                      int64
Department Name                   object
Latitude                         float64
Longitude                        float64
Market          

In [7]:
df['order date (DateOrders)'] = pd.to_datetime(df['order date (DateOrders)'], format='%m/%d/%Y %H:%M')

In [19]:
df['shipping date (DateOrders)']

0          2/3/2018 22:56
1         1/18/2018 12:27
2         1/17/2018 12:06
3         1/16/2018 11:45
4         1/15/2018 11:24
               ...       
180514     1/20/2016 3:40
180515     1/19/2016 1:34
180516    1/20/2016 21:00
180517    1/18/2016 20:18
180518    1/19/2016 18:54
Name: shipping date (DateOrders), Length: 180519, dtype: object

In [27]:
# Define the date to filter from
filter_date = pd.Timestamp('2017-10-01')

# Filter the DataFrame for dates greater than October 2017
filtered_df = df[df['order date (DateOrders)'] > filter_date]

In [28]:
fl_df = df[df['order date (DateOrders)'] <= filter_date]

In [37]:
fl_df.to_csv(r'C:\Users\Administrator\Desktop\KPIM\Internship Folder\Dataset\On - primises DW\data iload\full_load.csv')

In [21]:
# Create a dictionary to hold the DataFrames for each month
monthly_dfs = {}

# Iterate over the unique year-month combinations
for year_month, group in filtered_df.groupby(filtered_df['order date (DateOrders)'].dt.to_period('M')):
    # Convert the period to string and create a DataFrame for each month
    monthly_dfs[str(year_month)] = group

# Format 'order date (DateOrders)' column in each DataFrame to 'MM/DD/YYYY HH:MM'
for key in monthly_dfs:
    monthly_dfs[key]['order date (DateOrders)'] = monthly_dfs[key]['order date (DateOrders)'].dt.strftime('%m/%d/%Y %H:%M')



In [25]:
import os

In [26]:
# Define the folder to save the files
output_folder = r"C:\Users\Administrator\Desktop\KPIM\Internship Folder\Dataset\On - primises DW\data iload"

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Save each DataFrame to a CSV file in the specified folder
for key in monthly_dfs:
    output_path = os.path.join(output_folder, f"{key}.csv")
    monthly_dfs[key].to_csv(output_path, index=False)

# Verify the result
print("Files have been saved successfully.")

Files have been saved successfully.
